In [1]:
from pymongo import MongoClient
from lxml import html

import os
import requests
import time

mongo_connection_uri = os.environ.get("ME_CONFIG_MONGODB_URL")
client = MongoClient(mongo_connection_uri)

db = client["atcoder"]
contest_tasks_raw_collection = db["contest_tasks_raw"]
task_info_raw_collection = db["task_info_raw"]

In [2]:
def get_html(task_url):
    url = f"https://atcoder.jp{task_url}"
    cookies = {
        "_ga": "GA1.1.1669735030.1727884004",
        "REVEL_FLASH": "",
        "timeDelta": "1798",
        "REVEL_SESSION": "c78a68c12e880e173a54d016fe6631dba6a73914-%00UserName%3Adarcy_zhang%00%00a%3Afalse%00%00w%3Afalse%00%00SessionKey%3Af712625f246d5b11787eeeac74b6cefd3f43ebfeb6e05e5a6b9568a08ba4367cebdfbc32acea8a-0b0fdeaa544c89f44b7326075ad509e6ba62c1c71fa969e5d0cab3c4e61ee30d%00%00UserScreenName%3Adarcy_zhang%00%00csrf_token%3AiecE%2FCD%2BWkZctrj%2BaFUUE0TovINExsqeelroJ6VJx9Q%3D%00%00_TS%3A1746185522%00",
        "_ga_RC512FD18N": "GS1.1.1730628353.9.1.1730633519.0.0.0",
    }

    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,en-GB;q=0.6,zh-TW;q=0.5",
        "cache-control": "no-cache",
        "pragma": "no-cache",
        "priority": "u=0, i",
        "referer": "https://atcoder.jp/contests/abc378/tasks",
        "sec-ch-ua": '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0",
    }

    response = requests.get(url, cookies=cookies, headers=headers)
    response.encoding = "utf-8"
    return response.text

In [3]:
for contest_tasks_raw_item in contest_tasks_raw_collection.find():
    tree = html.fromstring(contest_tasks_raw_item["raw_html"])
    tbody_list = tree.xpath("//tbody")
    if len(tbody_list) != 1:
        continue
    tr_list = tree.xpath("//tbody/tr")
    for tr_elem in tr_list:
        time.sleep(0.3)
        task_url = tr_elem.xpath(".//td[2]/a")[0].get("href")
        existing_task = task_info_raw_collection.find_one({"task_url": task_url})
        if existing_task is None:
            raw_html = get_html(task_url)
            task_info_raw_collection.insert_one(
                {"task_url": task_url, "raw_html": raw_html}
            )
        else:
            print(f"Task with URL {task_url} already exists, skipping insertion.")

Task with URL /contests/abc378/tasks/abc378_a already exists, skipping insertion.
Task with URL /contests/abc378/tasks/abc378_b already exists, skipping insertion.
Task with URL /contests/abc378/tasks/abc378_c already exists, skipping insertion.
Task with URL /contests/abc378/tasks/abc378_d already exists, skipping insertion.
Task with URL /contests/abc378/tasks/abc378_e already exists, skipping insertion.
Task with URL /contests/abc378/tasks/abc378_f already exists, skipping insertion.
Task with URL /contests/abc378/tasks/abc378_g already exists, skipping insertion.
Task with URL /contests/arc186/tasks/arc186_a already exists, skipping insertion.
Task with URL /contests/arc186/tasks/arc186_b already exists, skipping insertion.
Task with URL /contests/arc186/tasks/arc186_c already exists, skipping insertion.
Task with URL /contests/arc186/tasks/arc186_d already exists, skipping insertion.
Task with URL /contests/arc186/tasks/arc186_e already exists, skipping insertion.
Task with URL /c